<a href="https://colab.research.google.com/github/olexandr7/numismatic_data_tallinn_city_museum/blob/main/numismatic_data_muis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import xml.etree.ElementTree as ET
import pandas as pd

# Path to your XML file
xml_file = "lido.xml"
ns = {"lido": "http://www.lido-schema.org"}

# Parse the XML
tree = ET.parse(xml_file)
root = tree.getroot()

In [42]:
rows = []
for record in root.findall("lido:lido", ns):
    recid = record.findtext(".//lido:lidoRecID[@lido:type='local']", namespaces=ns)
    muuseumikogu = record.findtext(".//lido:classification[@lido:type='muuseumikogu']/lido:term", namespaces=ns)
    number = record.findtext(".//lido:workID[@lido:type='museaali täisnumber']", namespaces=ns)
    nimetus = record.findtext(".//lido:titleSet[@lido:type='museaali nimetus']/lido:appellationValue", namespaces=ns)
    olemus = record.findtext(".//lido:objectWorkType[@lido:type='olemus']/lido:term", namespaces=ns)
    seisund = record.findtext(".//lido:displayStateEditionWrap/lido:displayState", namespaces=ns)
    date_start = record.findtext(".//lido:eventDate/lido:date/lido:earliestDate", namespaces=ns)
    date_end = record.findtext(".//lido:eventDate/lido:date/lido:latestDate", namespaces=ns)
    dateering = f"{date_start}–{date_end}" if date_start and date_end else (date_start or date_end)
# --- Originaal + Originaal_tüüp
    desc = record.findtext(".//lido:descriptiveNoteValue", namespaces=ns)
    originaal = desc
    originaal_tüüp = (
        "koopia"
        if desc and "koopia" in desc.lower()
        else "originaal" if desc
        else None
    )
# --- Tehnika: keep only actual technique(s) after skipping category + subcategory ---
    tehnika_terms = []
    for block in record.findall(".//lido:termMaterialsTech[@lido:type='tehnika']", ns):
        terms = [t.text.strip() for t in block.findall("lido:term", ns) if t.text]
        if len(terms) > 2:
            tehnika_terms.extend(terms[2:])  # e.g., ['vermimine']
        elif len(terms) > 1:
            tehnika_terms.extend(terms[1:])
    tehnika = "; ".join(dict.fromkeys(tehnika_terms))  # de-duplicate, keep order

# --- Materjal: keep only the most specific (last) term per block ---
    materjal_terms = []
    for block in record.findall(".//lido:termMaterialsTech[@lido:type='materjal']", ns):
        terms = [t.text.strip() for t in block.findall("lido:term", ns) if t.text]
        if terms:
            materjal_terms.append(terms[-1])  # e.g., 'kollane metall'
    materjal = "; ".join(dict.fromkeys(materjal_terms))  # de-duplicate, keep order
#-------------------------------------------------------------
    mõõdud_type = record.findtext(".//lido:measurementType", namespaces=ns)
    mõõdud_val = record.findtext(".//lido:measurementValue", namespaces=ns)
    mõõdud_unit = record.findtext(".//lido:measurementUnit", namespaces=ns)
    mõõdud = f"{mõõdud_type} {mõõdud_val} {mõõdud_unit}" if mõõdud_type and mõõdud_val and mõõdud_unit else None

# --- Kommentaar (sourceDescriptiveNote)
    kommentaar = record.findtext(".//lido:sourceDescriptiveNote", namespaces=ns)
#-------------------------------------------------------------
    link = f"https://www.muis.ee/museaalview/{recid}" if recid else None


    rows.append({
        "RecID": recid,
        "Link": link,
        "Muuseumikogu": muuseumikogu,
        "Number": number,
        "Nimetus": nimetus,
        "Olemus": olemus,
        "Dateering": dateering,
        "Originaal": originaal,
        "Originaal_tüüp": originaal_tüüp,
        "Seisund": seisund,
        "Tehnika": tehnika,
        "Materjal": materjal,
        "Mõõdud": mõõdud,
        "Kommentaar": kommentaar
    })

# Create DataFrame
df = pd.DataFrame(rows)


In [40]:
print(df)

      RecID                                     Link  Muuseumikogu  \
0   1201021  https://www.muis.ee/museaalview/1201021  numismaatika   
1   1202822  https://www.muis.ee/museaalview/1202822  numismaatika   
2   1202821  https://www.muis.ee/museaalview/1202821  numismaatika   
3   1200131  https://www.muis.ee/museaalview/1200131  numismaatika   
4   1188609  https://www.muis.ee/museaalview/1188609  numismaatika   
5   1569273  https://www.muis.ee/museaalview/1569273  numismaatika   
6   1569440  https://www.muis.ee/museaalview/1569440  numismaatika   
7   1569459  https://www.muis.ee/museaalview/1569459  numismaatika   
8   1199309  https://www.muis.ee/museaalview/1199309  numismaatika   
9   1569464  https://www.muis.ee/museaalview/1569464  numismaatika   
10  1197479  https://www.muis.ee/museaalview/1197479  numismaatika   
11  1197540  https://www.muis.ee/museaalview/1197540  numismaatika   
12  1240837  https://www.muis.ee/museaalview/1240837  numismaatika   
13  1201935  https:/

In [ ]:
# Save automatically to CSV
output_file = "muis_records.csv"
df.to_csv(output_file, index=False, encoding="utf-8-sig")

In [43]:
print(df.iloc[0])

RecID                                                     1201021
Link                      https://www.muis.ee/museaalview/1201021
Muuseumikogu                                         numismaatika
Number                                              TLM _ 27660 N
Nimetus                                                     tukat
Olemus                                                       münt
Dateering                                               1990–2000
Originaal       Keskaegse kuldtukati koopia, üle kullatud raud...
Seisund                                                       hea
Tehnika                                                 vermimine
Materjal                                           kollane metall
Mõõdud                                            läbimõõt 2.4 cm
Name: 0, dtype: object
